In [1]:
%%capture
!pip install --upgrade -tensorflow_hub
# !pip install -U -huggingface_hub

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="4"

import textattack
import transformers
import torch
import time
from datasets import Dataset
import sys
import hashlib
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertForMaskedLM, pipeline
from textattack.attack_recipes import (
    TextBuggerLi2018, DeepWordBugGao2018, TextFoolerJin2019, BERTAttackLi2020
)
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.models.wrappers import ModelWrapper

sys.path.append('../')
from eval_utils import *
sys.path.pop()

2023-08-05 17:39:50.764435: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-05 17:39:51.850861: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# set a seed, because reproducability is cool
np.random.seed(int(hashlib.sha256('Harrison Gietz'.encode('utf-8')).hexdigest(), 16) % 2**32)
torch.cuda.empty_cache()

device = input('enter a device name to run on: ')
dataset_val = input('Enter the number of samples to run on (100 or 1000): ')
defense = input('Specify a defense type among "default", "logit", "maj_log", "one_hot": ')

ag_tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-ag-news")
ag_model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-ag-news")
ag_model.to(device)
ag_pipeline = pipeline('sentiment-analysis', model=ag_model, tokenizer=ag_tokenizer)
ag_pipeline.device = next(ag_model.parameters()).device

ag_model_directory = "../../../models/bert-uncased_maskedlm_agnews_july31" #first diff
finetuned_ag_maskedlm = BertForMaskedLM.from_pretrained(ag_model_directory)
finetuned_ag_maskedlm.to(device)
ag_fill_mask = pipeline("fill-mask", model=finetuned_ag_maskedlm, tokenizer=ag_tokenizer)
ag_fill_mask.device = next(ag_model.parameters()).device

num_voter = 11
mask_pct = 0.3    
    
attack = DeepWordBugGao2018

if dataset_val == '100':
    loaded_ag_100 = Dataset.load_from_disk('../data/filtered_ag_clean_100')
    ag_100 = textattack.datasets.Dataset(convert_to_tuples(loaded_ag_100))
    dataset = ag_100
    dataset_name = 'ag-news100'
elif dataset_val =='1000':
    loaded_ag_1000 = Dataset.load_from_disk('../data/filtered_ag_clean_1000')
    ag_1000 = textattack.datasets.Dataset(convert_to_tuples(loaded_ag_1000))
    dataset = ag_1000
    dataset_name = 'ag-news1000'
else:
    raise ValueError('Number of samples not supported')
    
if defense == "default":
    ag_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(ag_model, ag_tokenizer)
    print(ag_wrapper)
elif defense == "logit":
    ag_wrapper = MaskDemaskWrapper(ag_model, ag_tokenizer, ag_fill_mask, num_voter, mask_pct, 'logit')
elif defense == 'maj_log':
    ag_wrapper = MaskDemaskWrapper(ag_model, ag_tokenizer, ag_fill_mask, num_voter, mask_pct, 'maj_log')
elif defense == "one_hot":
    ag_wrapper = MaskDemaskWrapper(ag_model, ag_tokenizer, ag_fill_mask, num_voter, mask_pct, 'maj_one_hot')
else:
    raise ValueError('Not a valid defense type.')
    
print(f'using num_voter = {num_voter} and mask_pct = {mask_pct} with dataset = {dataset_name}...')

# Parse the attack name
attack_name = parse_attack_name(attack)
attack = attack.build(ag_wrapper)

# Set up arguments for the attack
attack_args = textattack.AttackArgs(
    num_examples=len(dataset),
    log_to_csv=f'{attack_name}_{dataset_name}_mp{mask_pct}_nv{num_voter}_{defense}_log.csv',
    checkpoint_interval=25, 
    checkpoint_dir="chkpts_2", 
    disable_stdout=True
)
# Perform the attack and save the results
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

print(f'The above are results for {attack_name}_{dataset_name}_mp{mask_pct}_nv{num_voter}_{defense}.')

enter a device name to run on: cuda:0
Enter the number of samples to run on (100 or 1000): 1000
Specify a defense type among "default", "logit", "maj_log", "one_hot": maj_log
using num_voter = 11 and mask_pct = 0.3 with dataset = ag-news1000...
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterSubstitution(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (3): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    )
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  30
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
[Succeeded / Failed / Skipped / Total] 4 / 19 / 2 / 25:   2%|▎         | 25/1000 [26:42<17:21:40, 64.10s/it]

[Succeeded / Failed / Skipped / Total] 8 / 38 / 4 / 50:   5%|▌         | 50/1000 [48:20<15:18:38, 58.02s/it]

[Succeeded / Failed / Skipped / Total] 16 / 53 / 6 / 75:   8%|▊         | 75/1000 [1:06:49<13:44:16, 53.47s/it]

[Succeeded / Failed / Skipped / Total] 19 / 72 / 9 / 100:  10%|█         | 100/1000 [1:27:22<13:06:22, 52.43s/it]

...

[Succeeded / Failed / Skipped / Total] 159 / 752 / 39 / 950:  95%|█████████▌| 950/1000 [28:31:47<1:30:05, 108.11s/it]

[Succeeded / Failed / Skipped / Total] 164 / 771 / 40 / 975:  98%|█████████▊| 975/1000 [29:21:06<45:09, 108.38s/it]  

[Succeeded / Failed / Skipped / Total] 167 / 792 / 41 / 1000: 100%|██████████| 1000/1000 [30:12:30<00:00, 108.75s/it]

[Succeeded / Failed / Skipped / Total] 167 / 792 / 41 / 1000: 100%|██████████| 1000/1000 [30:12:31<00:00, 108.75s/it]



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 167    |
| Number of failed attacks:     | 792    |
| Number of skipped attacks:    | 41     |
| Original accuracy:            | 95.9%  |
| Accuracy under attack:        | 79.2%  |
| Attack success rate:          | 17.41% |
| Average perturbed word %:     | 7.94%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 121.93 |
+-------------------------------+--------+
The above are results for DeepWordBugGao2018_ag-news1000_mp0.3_nv11_maj_log.
